In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price-prediction-seattle/train.csv
/kaggle/input/house-price-prediction-seattle/test.csv


Open .csv and remove entries without lot sizes.

In [2]:
train_df = pd.read_csv("/kaggle/input/house-price-prediction-seattle/train.csv")
clean_train_df = train_df[(train_df["lot_size_units"] == "sqft") | (train_df["lot_size_units"] == "acre")]

Convert all lot_size units to sqft

In [3]:
lot_sizes = np.where(clean_train_df["lot_size_units"] == "acre", clean_train_df["lot_size"]*43560, clean_train_df["lot_size"])

Extract features and targets into X and y

In [4]:
X = np.column_stack(
    [
        lot_sizes, 
        clean_train_df["beds"], 
        clean_train_df["baths"], 
        clean_train_df["size"]
    ]
)
y = clean_train_df["price"]

Using models based on guidance here: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

Trying Lasso, changing alpha has no effect

In [5]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
lasso_model = Lasso(alpha=0.01)
lasso_model.fit(X,y)

lasso_predicted = lasso_model.predict(X)
lasso_metrics = mean_squared_error(y,lasso_predicted), lasso_model.score(X,y)

Trying with ElasticNet, changing alpha has no effect

In [6]:
from sklearn.linear_model import ElasticNet
elasticnet_model = ElasticNet(alpha=0.01)
elasticnet_model.fit(X,y)

elasticnet_predicted = elasticnet_model.predict(X)
elasticnet_metrics = mean_squared_error(y,elasticnet_predicted), elasticnet_model.score(X,y)

Trying SVR
- kernel="linear" performs about the same as previous
- kernel="poly"
    - degree=3 performs notably worse than others
    - degree=2 also performs poorly, poly is not the right direction
- kernel="rbf" also performs poorly, doesn't seem right

In [7]:
from sklearn.svm import SVR
svr_linear_model = SVR(kernel="linear")
svr_linear_model.fit(X,y)

svr_linear_predicted = svr_linear_model.predict(X)

svr_metrics = mean_squared_error(y,svr_linear_predicted), svr_linear_model.score(X,y)

Comparing performance across all three model types:
- Lasso + ElasticNet are almost identical
- SVR has higher MSE and lower coef. of determination

Lasso and ElasticNet are the way to go.

In [8]:
lasso_metrics, elasticnet_metrics, svr_metrics

((840690646928.8298, 0.17201239445698346),
 (840691649590.9625, 0.1720114069453209),
 (854844072849.7957, 0.15807283026492092))

Going to try taking np.log() of lot size feature to see if it helps with linear model: SUCCESS! - slightly better performance

In [9]:
X = np.column_stack(
    [
        np.log(lot_sizes), 
        clean_train_df["beds"], 
        clean_train_df["baths"], 
        clean_train_df["size"]
    ]
)

lasso_model = Lasso(alpha=0.001)
lasso_model.fit(X,y)
lasso_predicted = lasso_model.predict(X)
lasso_metrics = mean_squared_error(y,lasso_predicted)/1669, lasso_model.score(X,y)

elasticnet_model = ElasticNet(alpha=0.001)
elasticnet_model.fit(X,y)
elasticnet_predicted = elasticnet_model.predict(X)
elasticnet_metrics = mean_squared_error(y,elasticnet_predicted)/1669, elasticnet_model.score(X,y)

lasso_metrics, elasticnet_metrics

((503396939.05722076, 0.17252567531003327),
 (503396944.91512984, 0.17252566568091365))

Now I will use test data to see if the models are overfit
- if anything, they are underfit - I'm going to decrease alpha more for both

In [10]:
test_df = pd.read_csv("/kaggle/input/house-price-prediction-seattle/test.csv")
clean_test_df = test_df[(test_df["lot_size_units"] == "sqft") | (test_df["lot_size_units"] == "acre")]
test_lot_sizes = np.where(clean_test_df["lot_size_units"] == "acre", clean_test_df["lot_size"]*43560, clean_test_df["lot_size"])

test_X = np.column_stack(
    [
        np.log(test_lot_sizes), 
        clean_test_df["beds"], 
        clean_test_df["baths"], 
        clean_test_df["size"]
    ]
)

test_y = clean_test_df["price"]

lasso_test_predicted = lasso_model.predict(test_X)
lasso_test_metrics = mean_squared_error(test_y,lasso_test_predicted)/428, lasso_model.score(test_X,test_y)

elasticnet_test_predicted = elasticnet_model.predict(test_X)
elasticnet_test_metrics = mean_squared_error(test_y,elasticnet_test_predicted)/428, elasticnet_model.score(test_X,test_y)

lasso_test_metrics, elasticnet_test_metrics

((451819988.76272845, 0.4940302650416619),
 (451823191.57973474, 0.49402667837327163))

Somewhat happy with results, so:
1. I will get the coefficients for both models
2. I will calculate the outputs separately
3. I will average the outputs for a final answer

In [11]:
elasticnet_model.coef_, elasticnet_model.intercept_

(array([ -24177.07674399, -125209.39370136,   61891.57328726,
            520.31715415]),
 506827.50826334255)

In [12]:
lasso_model.coef_, lasso_model.intercept_

(array([ -24183.56640392, -125325.16519638,   61970.75214076,
            520.37275021]),
 506956.92602806364)

Based on the coefficients, lot size and number of beds are negatively weighted, which seems wrong. This suggests that these four features don't accurately capture the data. I think zip code might be an important factor. Maybe I could figure out the ranking of the dataset's zip codes in terms of "niceness" and map them to an order for the model to work with.

Testing out zip code feature, seems to be informative

In [13]:
import plotly.express as px
px.scatter_3d(x=train_df["zip_code"], y=train_df["size"], z=train_df["price"], log_y=True)

Mapping zip codes to be more useful

In [14]:
unique_zip_codes = np.unique(np.concatenate((clean_train_df["zip_code"], clean_test_df["zip_code"])))
unique_zip_codes

array([98101, 98102, 98103, 98104, 98105, 98106, 98107, 98108, 98109,
       98112, 98115, 98116, 98117, 98118, 98119, 98121, 98122, 98125,
       98126, 98133, 98136, 98144, 98146, 98168, 98177, 98178, 98188,
       98199])

In [15]:
zip_code_mapping = {}
count = 1
for zip_code in unique_zip_codes:
    zip_code_mapping[zip_code] = count
    count += 1
    
mapped_zip_codes = np.array([zip_code_mapping[zip_code] for zip_code in clean_train_df["zip_code"]])

Recreating feature set with zip codes included

In [16]:
X = np.column_stack(
    [
        mapped_zip_codes,
        clean_train_df["beds"], 
        clean_train_df["baths"],
        clean_train_df["size"]
    ]
)

y = clean_train_df["price"]

lasso_model = Lasso(alpha=0.00001)
lasso_model.fit(X,y)
lasso_predicted = lasso_model.predict(X)
lasso_metrics = mean_squared_error(y,lasso_predicted)/1669, lasso_model.score(X,y)

elasticnet_model = ElasticNet(alpha=0.00001)
elasticnet_model.fit(X,y)
elasticnet_predicted = elasticnet_model.predict(X)
elasticnet_metrics = mean_squared_error(y,elasticnet_predicted)/1669, elasticnet_model.score(X,y)

lasso_metrics, elasticnet_metrics

((497222558.62159485, 0.1826750045668165),
 (497222558.62208194, 0.1826750045660157))

In [17]:
mapped_test_zip_codes = np.array([zip_code_mapping[zip_code] for zip_code in clean_test_df["zip_code"]])

In [18]:
test_X = np.column_stack(
    [
        mapped_test_zip_codes, 
        clean_test_df["beds"], 
        clean_test_df["baths"],
        clean_test_df["size"]
    ]
)

test_y = clean_test_df["price"]

lasso_test_predicted = lasso_model.predict(test_X)
lasso_test_metrics = mean_squared_error(test_y,lasso_test_predicted)/428, lasso_model.score(test_X,test_y)

elasticnet_test_predicted = elasticnet_model.predict(test_X)
elasticnet_test_metrics = mean_squared_error(test_y,elasticnet_test_predicted)/428, elasticnet_model.score(test_X,test_y)

lasso_test_metrics, elasticnet_test_metrics

((454752683.84941363, 0.4907460921572636),
 (454752768.6154244, 0.49074599723221135))

In [19]:
elasticnet_model.coef_, elasticnet_model.intercept_

(array([ -14776.02979998, -115466.1605125 ,   54288.46503696,
            516.18479445]),
 502062.72389276454)

In [20]:
lasso_model.coef_, lasso_model.intercept_

(array([ -14776.01915874, -115467.22508706,   54289.19881478,
            516.18529419]),
 502063.2705224606)